In [1]:
import matplotlib.pyplot as plt  
import scipy
from scipy.stats import norm
from collections import Counter
from ipywidgets import interactive, IntSlider

In [2]:
def read_sequence(filename):
    with open(filename, "r") as file, open(output_file, "w") as sequences: 
        file_content = file.readlines()
        for i in range(len(file_content)):
            if file_content[i].startswith("@"):
                sequences.writelines(file_content[i+1])
    
    return output_file

In [3]:
def kmer_frequency(output_file, length):
    output_dict = {}
    with open(output_file, 'r') as sequences:
        for line in sequences:
            # Save the kmer frequency for each sequence
            line_dict = {}
            for i in range(0,len(line)-length+1):
                fragment = line[i:i+length]
                count = line.count(fragment)
                line_dict[fragment] = count
            
            # Add the kmer frequency in each sequence to the total dictionary 
            for key in line_dict:
                if key in output_dict.keys():
                    output_dict[key] = output_dict[key] + line_dict[key]
                else:
                    output_dict[key] = line_dict[key]

    return output_dict

In [4]:
def bar_interactive(length):
    output_file = read_sequence(filename)
    output = kmer_frequency(output_file,length)
    top10 = dict(Counter(output).most_common(10))
    
    #Plot the bar chart for top 10 values
    fig1 = plt.figure(figsize = (10,5))
    ax = fig1.add_subplot(1,1,1)
    ax.bar(list(top10.keys()), top10.values(), color = "b")
    ax.set_xlabel("Fragments",fontsize = 15)
    ax.set_ylabel("Counts",fontsize = 15)

In [7]:
%matplotlib inline
filename = '/Users/apple/Desktop/Summer_project/dataset/ENCFF170YQV_rep2_1.fq'
output_file = "sequences.txt"
length = 6

# create interactive sliders for kmer sequence
length_widget = IntSlider(min = 1, max = 1000, value = length)

# adjust settings to prevent continous recalculation and update of plot while user drags widget
for item in [ length_widget ]:
    item.continuous_update = False

interactive(bar_interactive, length = length_widget)

interactive(children=(IntSlider(value=6, continuous_update=False, description='length', max=1000, min=1), Outp…

In [8]:
%matplotlib inline
filename = '/Users/apple/Desktop/Summer_project/dataset/ENCFF239CML_rep1_1.fq'
output_file = "sequences.txt"
length = 6

# create interactive sliders for kmer sequence
length_widget = IntSlider(min = 1, max = 1000, value = length)

# adjust settings to prevent continous recalculation and update of plot while user drags widget
for item in [ length_widget ]:
    item.continuous_update = False

interactive(bar_interactive, length = length_widget)

interactive(children=(IntSlider(value=6, continuous_update=False, description='length', max=1000, min=1), Outp…

In [9]:
%matplotlib inline
filename = '/Users/apple/Desktop/Summer_project/dataset/ENCFF239CML_rep1_2.fq'
output_file = "sequences.txt"
length = 6

# create interactive sliders for kmer sequence
length_widget = IntSlider(min = 1, max = 1000, value = length)

# adjust settings to prevent continous recalculation and update of plot while user drags widget
for item in [ length_widget ]:
    item.continuous_update = False

interactive(bar_interactive, length = length_widget)

interactive(children=(IntSlider(value=6, continuous_update=False, description='length', max=1000, min=1), Outp…

In [10]:
%matplotlib inline
filename = '/Users/apple/Desktop/Summer_project/dataset/ENCFF170YQV_rep2_2.fq'
output_file = "sequences.txt"
length = 6

# create interactive sliders for kmer sequence
length_widget = IntSlider(min = 1, max = 1000, value = length)

# adjust settings to prevent continous recalculation and update of plot while user drags widget
for item in [ length_widget ]:
    item.continuous_update = False

interactive(bar_interactive, length = length_widget)

interactive(children=(IntSlider(value=6, continuous_update=False, description='length', max=1000, min=1), Outp…